# Demo of EMU generation for TCA cycle example

# Intro

# Setup

First, we need to set the path and environment variable properly:

In [1]:
quantmodelDir = '/users/hgmartin/libraries/quantmodel'

This is the only place where the jQMM library path needs to be set.

In [2]:
%matplotlib inline

import sys, os
pythonPath = quantmodelDir+"/code/core"
if pythonPath not in sys.path:
    sys.path.append(pythonPath)
os.environ["QUANTMODELPATH"] = quantmodelDir

In [3]:
%matplotlib inline
import os, core, FluxModels, unittest, enhancedLists
from IPython.display import SVG, Image

And move to a scratch directory (to make things faster):

In [4]:
cd /scratch/hgmartin_scratch/tests

/scratch/hgmartin_scratch/tests


# Assembling SBML file

Get file names needed to produce sbml file:

In [5]:
qmodeldir         = os.environ['QUANTMODELPATH']    
dirDATA           = qmodeldir+'/data/tests/TCAtoy/' 


REACTIONSfilename   = dirDATA+'REACTIONStca.txt' 
FEEDfilename        = dirDATA+'FEEDtca.txt'
CEMSfilename        = dirDATA+'GCMStca.txt'
CEMSSTDfilename     = dirDATA+'GCMSerrtca.txt'
FLUXESFreefilename  = dirDATA+'FLUXtca.txt'

Assemble sbml file:

In [6]:
atomTransitions = enhancedLists.AtomTransitionList(REACTIONSfilename)
ReacNet = atomTransitions.getReactionNetwork('E. coli wt5h 13C MFA')

ReacNet.addLabeling(CEMSfilename,'LCMSLabelData',CEMSSTDfilename,minSTD=0.001)
ReacNet.addFeed(FEEDfilename)
ReacNet.loadFluxBounds(FLUXESFreefilename)

ReacNet.write('TCA.sbml')  

1

# Obtaining EMU transitions

Load model:

In [7]:
C13model = FluxModels.C13Model('TCA.sbml') 

Get reaction list, EMU transitions and carbon transitions for later use:

In [8]:
rxnList           = C13model.reactionNetwork.reactionList
EMUTransitions    = rxnList.transitions
carbonTransitions = enhancedLists.AtomTransitionList([core.AtomTransition(line) for line in rxnList.rawEMUReactions])

In [9]:
for trans in carbonTransitions.transitions:
    print trans

co2Out	co2_c --> co2ext_c	a : a
r1	oac_c + accoa_c --> cit_c	abcd + ef : dcbfea
r2	cit_c --> akg_c + co2_c	abcdef : abcde + f
r3	akg_c --> glu_L_c	abcde : abcde
r4	akg_c --> suc_c + co2_c	abcde : (bcde;edcb) + a
r5	suc_c --> fum_c	abcd : (abcd;dcba)
r6	fum_c --> oac_c	abcd : (abcd;dcba)
r7	oac_c --> fum_c	abcd : (abcd;dcba)
r8	asp_c --> oac_c	abcd : abcd


Get the list of metabolites from reactionList:

In [10]:
metList = rxnList.getMetList()

and obtain the destination EMUs from there:

In [11]:
destEMUs = metList.getDestinationEMUs()
print destEMUs

['glu_L_c_1_2_3_4_5']


as well as the source metabolites:

In [12]:
sources = rxnList.EMUSourceMetabs
print sources

['accoa_c', 'asp_c']


With the destination emus and the metabolite sources, we can obtain all EMU transitions from sources to destination (first number is the amount of flux going through that emu transition, 0.5 for symmetric molecules): 

In [13]:
carbonTransitions.findEMUTransitions2Source(core.EMU(destEMUs[0]),sources,cleanCache=True)

[(1.0, 'r3, akg_c_1_2_3_4_5 --> glu_L_c_1_2_3_4_5'),
 (1.0, 'r2, cit_c_1_2_3_4_5 --> akg_c_1_2_3_4_5'),
 (1.0, 'r1, accoa_c_1_2 + oac_c_2_3_4 --> cit_c_1_2_3_4_5'),
 (0.5, 'r6, fum_c_1_2_3 --> oac_c_2_3_4'),
 (0.5, 'r6, fum_c_2_3_4 --> oac_c_2_3_4'),
 (0.5, 'r5, suc_c_2_3_4 --> fum_c_2_3_4'),
 (0.5, 'r5, suc_c_1_2_3 --> fum_c_2_3_4'),
 (0.5, 'r4, akg_c_2_3_4 --> suc_c_2_3_4'),
 (0.5, 'r4, akg_c_3_4_5 --> suc_c_2_3_4'),
 (1.0, 'r2, cit_c_3_4_5 --> akg_c_3_4_5'),
 (1.0, 'r1, accoa_c_1_2 + oac_c_2 --> cit_c_3_4_5'),
 (0.5, 'r6, fum_c_2 --> oac_c_2'),
 (0.5, 'r6, fum_c_3 --> oac_c_2'),
 (0.5, 'r5, suc_c_3 --> fum_c_2'),
 (0.5, 'r5, suc_c_2 --> fum_c_2'),
 (0.5, 'r4, akg_c_3 --> suc_c_2'),
 (0.5, 'r4, akg_c_4 --> suc_c_2'),
 (1.0, 'r2, cit_c_3 --> akg_c_3'),
 (1.0, 'r1, oac_c_2 --> cit_c_3'),
 (1.0, 'r8, asp_c_2 --> oac_c_2'),
 (1.0, 'r2, cit_c_4 --> akg_c_4'),
 (1.0, 'r1, accoa_c_2 --> cit_c_4'),
 (0.5, 'r4, akg_c_3 --> suc_c_3'),
 (0.5, 'r4, akg_c_4 --> suc_c_3'),
 (0.5, 'r7, oac_c_3 --> 

# EMU equations

The EMU equations are divide in two parts: non-condensation equations and condensation equations.

The non-condensation equations are coded in the EMM matrix:

In [14]:
emmPar  = EMUTransitions.getEMMmat()

In [15]:
print emmPar.elements

{('r5', 'suc_c_1_2_3', 'fum_c_1_2_3'): 0.5, ('r2', 'akg_c_3', 'cit_c_3'): -1.0, ('r8', 'oac_c_3', 'asp_c_3'): -1.0, ('r8', 'asp_c_1_2_3', 'oac_c_1_2_3'): 1.0, ('r5', 'fum_c_3', 'suc_c_2'): -0.5, ('r7', 'oac_c_1_2_3', 'fum_c_1_2_3'): 0.5, ('r1', 'accoa_c_1_2oac_c_2_3_4', 'cit_c_1_2_3_4_5'): 1.0, ('r8', 'asp_c_2_3_4', 'oac_c_2_3_4'): 1.0, ('r4', 'suc_c_2', 'akg_c_4'): -0.5, ('r3', 'akg_c_1_2_3_4_5', 'glu_L_c_1_2_3_4_5'): 1.0, ('r6', 'oac_c_2_3', 'fum_c_2_3'): -1.0, ('r5', 'suc_c_1_2_3', 'fum_c_2_3_4'): 0.5, ('r1', 'accoa_c_2oac_c_2_3', 'cit_c_2_3_4'): 1.0, ('r6', 'fum_c_2_3', 'oac_c_2_3'): 1.0, ('r8', 'oac_c_2_3', 'asp_c_2_3'): -1.0, ('r5', 'suc_c_2_3', 'fum_c_2_3'): 1.0, ('r4', 'suc_c_1_2_3', 'akg_c_3_4_5'): -0.5, ('r5', 'suc_c_3', 'fum_c_3'): 0.5, ('r7', 'fum_c_2', 'oac_c_3'): -0.5, ('r4', 'suc_c_3', 'akg_c_4'): -0.5, ('r4', 'akg_c_3_4_5', 'suc_c_1_2_3'): 0.5, ('r8', 'oac_c_1_2_3', 'asp_c_1_2_3'): -1.0, ('r4', 'akg_c_3_4', 'suc_c_2_3'): 1.0, ('r1', 'cit_c_3_4_5', 'accoa_c_1_2oac_c_2'):

The condensation equations are coded in GAMS lines:

In [16]:
eqns,eqnNames = EMUTransitions.getCondGAMSReacs()

In [17]:
print eqns[0]

eqn_condensation_rxn_1at0..f('accoa_c_1_2oac_c_2','0') =e= f('accoa_c_1_2','0')*f('oac_c_2','0');

